# ID3 Decision Tree

<center><img src="https://i0.wp.com/dataaspirant.com/wp-content/uploads/2017/01/B03905_05_01-compressor.png?resize=768%2C424" width=50%></center>

## The ID3 Algorithm

### Entropy

**Entropy:** *a thermodynamic quantity representing the unavailability of a system's thermal energy for conversion into mechanical work, often interpreted as the degree of disorder or randomness in the system.*

$$
Entropy(S) = \sum_{i=1}^{c} -p_{i}\log2(p_{i})
$$

### Information Gain

- The information gain must be calculated for each feature in the set $S$;
- The feature with the highest information gain is selected to be the test feature;
- The information gain refers to the expected entropy reduction as the dataset is being partitioned.

$$
Information Gain(S, F) \equiv Entropy(S) - \sum_{f \in Features(F)} \frac{|Sv|}{|S|} Entropy(Sv)
$$

## Implementation

### Import libraries

In [1]:
from igraph import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cairocffi
%matplotlib inline

### Importing and visualizing the dataset

In [2]:
dataset = pd.read_csv("datasets/play_tennis.csv")

X = dataset.iloc[:, 1:-1].values
Y = dataset.iloc[:, -1].values.reshape((-1,1))

print("Datset shape:", X.shape)
print("Labels shape:", Y.shape)
print()

dataset

Datset shape: (14, 4)
Labels shape: (14, 1)



,day,weather,temperature,humidity,wind,play_tennis
0,D1,sunny,hot,high,weak,no
1,D2,sunny,hot,high,strong,no
2,D3,cloudy,hot,high,weak,yes
3,D4,rain,warm,high,weak,yes
4,D5,rain,cold,normal,weak,yes
5,D6,rain,cold,normal,strong,no
6,D7,cloudy,cold,normal,strong,yes
7,D8,sunny,warm,high,weak,no
8,D9,sunny,cold,normal,weak,yes
9,D10,rain,warm,normal,weak,yes


### Calulating the entropy of a given feature

In [3]:
def E(dataset, col = -1):
  '''
  If no column is given, this function will assume
  that the label column will be at the end
  '''
  sel_func = dataset.loc if type(col) == type("str") else dataset.iloc
  probs = sel_func[:, col].value_counts(sort = False).values / sel_func[:, col].count()
  return np.sum(-probs * np.log2(probs))

E(dataset)

0.9402859586706309

### Calculating the information gain

In [4]:
sep = "--------------------------------------------------"

print("Total number of rows:", dataset.loc[:, "weather"].count())
print()

print(sep)
col_val = "sunny"
nyes = dataset.loc[(dataset["weather"] == col_val) & (dataset["play_tennis"] == "yes")]["weather"].count()
nrows = dataset.loc[dataset["weather"] == col_val]["weather"].count()
print("Number of rows:", nrows)
print("yes/no: {0}/{1}".format(nyes, nrows - nyes))
print(dataset.loc[dataset["weather"] == col_val])
print(sep)
col_val = "cloudy"
nyes = dataset.loc[(dataset["weather"] == col_val) & (dataset["play_tennis"] == "yes")]["weather"].count()
nrows = dataset.loc[dataset["weather"] == col_val]["weather"].count()
print("Number of rows:", nrows)
print("yes/no: {0}/{1}".format(nyes, nrows - nyes))
print(dataset.loc[dataset["weather"] == col_val])
print(sep)
col_val = "rain"
nyes = dataset.loc[(dataset["weather"] == col_val) & (dataset["play_tennis"] == "yes")]["weather"].count()
nrows = dataset.loc[dataset["weather"] == col_val]["weather"].count()
print("Number of rows:", nrows)
print("yes/no: {0}/{1}".format(nyes, nrows - nyes))
print(dataset.loc[dataset["weather"] == col_val])

Total number of rows: 14

--------------------------------------------------
Number of rows: 5
yes/no: 2/3
    day weather temperature humidity    wind play_tennis
0    D1   sunny         hot     high    weak          no
1    D2   sunny         hot     high  strong          no
7    D8   sunny        warm     high    weak          no
8    D9   sunny        cold   normal    weak         yes
10  D11   sunny        warm   normal  strong         yes
--------------------------------------------------
Number of rows: 4
yes/no: 4/0
    day weather temperature humidity    wind play_tennis
2    D3  cloudy         hot     high    weak         yes
6    D7  cloudy        cold   normal  strong         yes
11  D12  cloudy        warm     high  strong         yes
12  D13  cloudy         hot   normal    weak         yes
--------------------------------------------------
Number of rows: 5
yes/no: 3/2
    day weather temperature humidity    wind play_tennis
3    D4    rain        warm     high    weak   

### Calculating the informatio gain manually

In [7]:
nrows = dataset.loc[:, "weather"].count()
probs = np.array([
    dataset.loc[dataset["weather"] == "sunny"]["weather"].count() / nrows,
    dataset.loc[dataset["weather"] == "cloudy"]["weather"].count() / nrows,
    dataset.loc[dataset["weather"] == "rain"]["weather"].count() / nrows
])
ESv = np.array([
    E(dataset.loc[dataset["weather"] == "sunny"]),
    E(dataset.loc[dataset["weather"] == "cloudy"]),
    E(dataset.loc[dataset["weather"] == "rain"])
])

print("Probs:", probs)
print("ESv: ", ESv)

E(dataset) - np.sum(probs*ESv)

Probs: [0.35714286 0.28571429 0.35714286]
ESv:  [0.97095059 0.         0.97095059]


0.2467498197744391

### Defining the functions to calculating the information gain

In [8]:
def filter_col(dataset, col):
  '''
  '''
  sel_func = dataset.loc if type(col) == type("str") else dataset.iloc
  fdss = []
  for col_val in sel_func[:, col].unique():
    fdss.append(sel_func[dataset[col] == col_val])
  return fdss
  
def IG(dataset, col, label_col = -1):
  '''
  '''
  sel_func = dataset.loc if type(col) == type("str") else dataset.iloc
  # The unique order is used to filter the columns
  # So this will be used to to avoid unwanted sorting
  vals = sel_func[:, col].unique()
  probs = sel_func[:, col].value_counts(sort = "False")[vals].values / sel_func[:, col].count()
  
  Es = E(dataset)
  
  fdss = filter_col(dataset, col)
  ESv = list(map(E, fdss))
  
  return Es - np.sum(probs * ESv)
  
print(IG(dataset, "weather"))
print(IG(dataset, "humidity"))
print(IG(dataset, "wind"))
print(IG(dataset, "temperature"))

0.2467498197744391
0.15183550136234136
0.04812703040826927
0.029222565658954647


### Testing the IGraph Package

Testing the IGrapth package to create the following decision tree

<img src="figures/decision_tree_model.png" width=40%>

In [2]:
g = Graph(directed = False)

g.add_vertex(name = "weather", label = "weather", color = "#e89519") #v0
g.add_vertex(name = "humidity", label = "humidity", color = "##1876e8") #v1

g.add_edges([(0,1)])

g.add_vertex(name = "wind", label = "wind", color = "#9b17e8")  #v2

g.add_edges([(0,2)])

g.add_vertex(label = "yes", color = "green") #v3

g.add_edges([(0,3)])

g.add_vertex(label = "no", color = "red") #v4

g.add_edges([(1,4)])

g.add_vertex(label = "yes", color = "green")  #v5

g.add_edges([(1,5)])

g.add_vertex(label = "no", color = "red") #v6

g.add_edges([(2,6)])

g.add_vertex(label = "yes", color="green") #v7

g.add_edges([(2,7)])

In [3]:
plot(g, layout = g.layout("tree"))

UnboundLocalError: local variable 'components' referenced before assignment

In [ ]:
!pip 